# Interfacing with the Eigen Library

In this notebook we will cover how to convert types used by the Eigen C++ Linear Algebra library between Python and C++. The only prerequisite for using this notebook is to complete all of the steps in the first, including downloading and installing the Eigen library.

To load the modules (written using C++ and **nanobind**) from the build directory, the following code must be run each time the kernel is restarted:

In [1]:
import sys, os
module_dir = os.path.abspath('build')
if module_dir not in sys.path:
    sys.path.append(module_dir)
    print("Directory 'build' has been added to Python's module path")

Directory 'build' has been added to Python's module path


## nb::ndarray class

Key to interfacing with Eigen (and other libraries such as NumPy, PyTorch, TensorFlow, JAX, and CuPy) is the `ndarray` type which is part of **nanobind** (in header `<nanobind/ndarray.h>`). This type is mapped to various other n-dimensional array types using pre-defined (custom) **bindings**, as explained in a previous notebook.

To gain an idea of the capabilities of this type (compared to NumPy's `np.array`), execute the following code:

In [10]:
%%writefile eigen1.cpp
#include <nanobind/ndarray.h>

namespace nb = nanobind;

NB_MODULE(eigen1, m) {
    m.def("inspect", [](const nb::ndarray<>& a) {
        nb::print(nb::str("Array data pointer : {}").format(a.data()));
        nb::print(nb::str("Array dimension : {}").format(a.ndim()));
        for (size_t i = 0; i < a.ndim(); ++i) {
            nb::print(nb::str("Array dimension {} : {}").format(i, a.shape(i)));
            nb::print(nb::str("Array stride    {} : {}").format(i, a.stride(i)));
        }
        nb::print(nb::str("Device ID = {} (cpu={}, cuda={})").format(a.device_id(),
            int(a.device_type() == nb::device::cpu::value),
            int(a.device_type() == nb::device::cuda::value)
        ));
        nb::print(nb::str("Array dtype: int16={}, uint32={}, float32={}").format(
            a.dtype() == nb::dtype<int16_t>(),
            a.dtype() == nb::dtype<uint32_t>(),
            a.dtype() == nb::dtype<float>()
        ));
    });
}

Overwriting eigen1.cpp


As always, build the relevant target with CMake:

In [11]:
!cmake --build build --target eigen1

[ 85%] Built target nanobind-static
[ 92%] Building CXX object CMakeFiles/eigen1.dir/eigen1.cpp.o
[100%] Linking CXX shared module eigen1.cpython-313-x86_64-linux-gnu.so
[100%] Built target eigen1


And try it out in Python:

In [12]:
import numpy as np
import eigen1

eigen1.inspect(np.array([[1,2,3], [3,4,5]], dtype=np.float32))

Array data pointer : <capsule object "nb_handle" at 0x7aa48c4513f0>
Array dimension : 2
Array dimension 0 : 2
Array stride    0 : 3
Array dimension 1 : 3
Array stride    1 : 1
Device ID = 0 (cpu=1, cuda=0)
Array dtype: int16=False, uint32=False, float32=True


Comparing the output with the definition of the function `inspect()` previously will give a good idea of the purpose of `nb::ndarray`. See the documentation for further details on how to create `np.array`s (and other Python libraries' types) objects from C++, and how to specify shape, strides etc. dynamically.

## Matrix operations with Eigen

Use of the Eigen library with **nanobind** can be thought of as a specialization of using the `nb::ndarray` class. The header `<nanobind/eigen/dense.h>` provides for conversions between Python and C++ (both ways) for types: `Eigen::Matrix`, `Eigen::Array`, `Eigen::Vector`, `Eigen::Ref`, `Eigen::Map`.

If a C++ function returns one of these types *by value*, **nanobind** will capture and wrap it in a NumPy array without making a copy. All other cases (returning by reference, returning an unevaluated expression template) either evaluate or copy the array.

If a C++ function accepts *by reference*, **nanobind** will still need to make a copy. To prevent this, use should be made of `nb::DRef` which makes the source data directly available to C++.

The following code is bad form for several reasons:

```plaintext
m.def("sum", [](Eigen::Vector3f a, Eigen::Vector3d b) { return a + b; }); // Bad!
```

Firstly, `a` and `b` are accepted by value (although see previous discussion of why by reference would be no better). Secondly, C++ would attempt to return the **unevaluated expression template** of `a + b` instead of a new `Eigen::Vector`. Thus the following should be used instead:

In [18]:
%%writefile eigen2.cpp
#include <nanobind/eigen/dense.h>

namespace nb = nanobind;

Eigen::Vector3f sum(const nb::DRef<Eigen::Vector3f> &a, const nb::DRef<Eigen::Vector3f> &b) {
    return a + b; // no need for (a + b).eval() as return type is specified
}

NB_MODULE(eigen2, m) {
    m.def("sum", &sum);
}

Overwriting eigen2.cpp


Build the same target with CMake:

In [19]:
!cmake --build build --target eigen2

[ 85%] Built target nanobind-static
[ 92%] Building CXX object CMakeFiles/eigen2.dir/eigen2.cpp.o
In file included from /home/debian/nanobind-tutorial/jupyter-notebooks/eigen-3.3.1/include/Eigen/Core:401,
                 from /home/debian/nanobind-tutorial/jupyter-notebooks/ext/nanobind/include/nanobind/eigen/dense.h:14,
                 from /home/debian/nanobind-tutorial/jupyter-notebooks/eigen2.cpp:1:
/home/debian/nanobind-tutorial/jupyter-notebooks/eigen-3.3.1/include/Eigen/src/Core/functors/StlFunctors.h:87:28: warning: ‘template<class _Predicate> class std::unary_negate’ is deprecated []8;;https://gcc.gnu.org/onlinedocs/gcc-14.2.0/gcc/Warning-Options.html#index-Wdeprecated-declarations-Wdeprecated-declarations]8;;]
   87 | struct functor_traits<std::unary_negate<T> >
      |                            ^~~~~~~~~~~~
In file included from /usr/include/c++/14/string:49,
                 from /usr/include/c++/14/stdexcept:39,
                 from /home/debian/nanobind-tutorial/j

Try it out in Python:

In [2]:
import numpy as np
import eigen2

print(eigen2.sum(np.array([1., 2., 3.], dtype='float32'),np.array([2., 3., 4.], dtype='float32')))

[3. 5. 7.]


If you get into difficulties with types, recall that by value implies possible type conversion. Use code such as `nb::arg("x").noconvert()` to avoid this.

*All text and program code &copy;2026 Richard Spencer, all rights reserved.*